In [ ]:
import tensorflow as tf
print(tf.__version__)

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

df = pd.read_csv("../input/text-data/dataset.csv")
df

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer

In [ ]:
sentences = list(df["Text"])
print(type(sentences))
print(len(sentences))

In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('stopwords')
stop_words = set(stopwords.words('spanish'))
print(stop_words)

In [ ]:
sentences_filter = []
for row in sentences:
    row = row.lower()
    tokenizer = nltk.RegexpTokenizer(r"\w+")
    new_words = tokenizer.tokenize(row)
    new_row = [word for word in new_words  if word not in stop_words]
    new_sentence = " ".join(new_row)
    sentences_filter.append(new_sentence)

print(sentences[0][:100])
print("Converted:")
print(sentences_filter[0][:100])

In [ ]:
tokenizer = Tokenizer(oov_token="<OOV>")
tokenizer.fit_on_texts(sentences_filter)
word_index = tokenizer.word_index
print(word_index)
total_words = list(word_index.keys())
print(len(total_words))

In [ ]:
from nltk import ngrams, FreqDist
full_text = []
for row in sentences_filter:
    full_text.extend(row.split(" "))
n_gramas = ngrams(total_words, n=1)
freq = FreqDist(full_text)
freq.plot(20)

In [ ]:
freq

In [ ]:
check_list = [] #filtramos tokens que se repiten mas de una vez
for key,val in freq.items():
    if (val==1):
        check_list.append(str(key))
        
print(check_list)

In [ ]:
final_sentences = []
other_stop_w = ["si", "usted", "señor", "gracias", "señorita"]
check_list.extend(other_stop_w)
for row in sentences_filter:
    words = row.split(" ")
    final_words = []
    for w in words:
        if w not in check_list:
            final_words.append(w)
    final_sentences.append(" ".join(final_words))
    
print(sentences_filter[0][:200])
print(final_sentences[0][:200])

In [ ]:
tokenizer = Tokenizer(oov_token="<OOV>")
tokenizer.fit_on_texts(final_sentences)
word_index = tokenizer.word_index
print(word_index)

In [ ]:
total_words = list(word_index.keys())
print(len(total_words))

In [ ]:
sequences = tokenizer.texts_to_sequences(final_sentences)
sequences_np = np.array(sequences)
mylen = np.vectorize(len)
seq_sizes = mylen(sequences_np)
print("Min sequence length:", np.min(seq_sizes))
print("Max sequence length:", np.max(seq_sizes))
print("Avg sequence length:", np.mean(seq_sizes))

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

embedding_dim = 300
max_length = 320
trunc_type='post'

padded = pad_sequences(sequences,maxlen=max_length, padding='post', truncating=trunc_type)

In [ ]:
text_file = "../input/pretrained-word-vectors-for-spanish/SBW-vectors-300-min5.txt"


data = []
with open(text_file, 'r') as fopen:
    for n, line in enumerate(fopen):
        data.append(line)
        if (n + 1) % 100000 == 0:
            print('done processed: ', n + 1)
            
print('done!')



In [ ]:
print(len(data))

In [ ]:
data = data[1:]

In [ ]:
vocab = total_words
import numpy
dict_words_vect = {}
for row in data:
        row = row[:-1]
        word = row.split(" ")[0]
        if word in vocab:
            vector = row.split(" ")[1:]
            vector = np.array([float(x) for x in vector])
            dict_words_vect[word] = vector

In [ ]:
print(len(vocab))
print(len(dict_words_vect))
print(dict_words_vect["tarjeta"].shape)

In [ ]:
words_in_dict = list(dict_words_vect.keys())
missing_words = list(set(vocab) - set(words_in_dict))
print(missing_words)
for word in missing_words:
    dict_words_vect[word] = np.zeros(300)

In [ ]:
print(dict_words_vect["netflix"].shape)

In [ ]:
vocab_size = len(vocab)
embedding_dim = 300

embeddings_matrix = np.zeros((vocab_size+1, embedding_dim));
for word, i in word_index.items():
    embedding_vector = dict_words_vect[word];
    if embedding_vector is not None:
        embeddings_matrix[i] = embedding_vector

In [ ]:
print(embeddings_matrix)
print(len(word_index))

In [ ]:
list_words = list(word_index.keys())
print(list_words[0])
print(list_words[-1])

In [ ]:
print(word_index["<OOV>"])
print(word_index["transfieren"])

In [ ]:
print(dict_words_vect["<OOV>"][:10])
print(dict_words_vect["transfieren"][:10])

In [ ]:
print(embeddings_matrix[0][:10])
print(embeddings_matrix[-1][:10])

In [ ]:
import json
import tensorflow as tf
import csv
import random
import numpy as np

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import regularizers

In [ ]:
df.describe()

In [ ]:
labels_producto = list(df["Producto"])
print(len(labels_producto))
n_classes = len(set(labels_producto))
print(n_classes)

In [ ]:
set(labels_producto)

In [ ]:
index_to_label_prod = {}
label_to_index_prod = {}
for i, label in enumerate(set(labels_producto)):
    index_to_label_prod[i] = label
    label_to_index_prod[label] = i
    
print(index_to_label_prod)
print(label_to_index_prod)

In [ ]:
labels_producto_num = np.array([label_to_index_prod[label] for label in labels_producto])
print(labels_producto_num)

In [ ]:
padded.shape

In [ ]:
split = int(len(padded)*0.9)
train_x = padded[:split]
valid_x = padded[split:]
train_y = labels_producto_num[:split]
valid_y = labels_producto_num[split:]
print(len(train_x))
print(len(train_y))

In [ ]:
model_prod = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size+1, embedding_dim, input_length=max_length, weights=[embeddings_matrix], trainable=True),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Conv1D(32, 5, activation='relu'), #64->32
    tf.keras.layers.MaxPooling1D(pool_size=4),
    tf.keras.layers.LSTM(32), #64->32
    tf.keras.layers.Dense(n_classes, activation='softmax')
])
model_prod.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
model_prod.summary()

num_epochs = 38

history = model_prod.fit(train_x, train_y, validation_data=(valid_x, valid_y), epochs=num_epochs)

print("Training Complete")

In [ ]:
import matplotlib.image  as mpimg
import matplotlib.pyplot as plt

def plot_h(history):

    #-----------------------------------------------------------
    # Retrieve a list of list results on training and test data
    # sets for each training epoch
    #-----------------------------------------------------------
    acc=history.history['accuracy']
    val_acc=history.history['val_accuracy']
    loss=history.history['loss']
    val_loss=history.history['val_loss']

    epochs=range(len(acc)) # Get number of epochs

    #------------------------------------------------
    # Plot training and validation accuracy per epoch
    #------------------------------------------------
    plt.plot(epochs, acc, 'r')
    plt.plot(epochs, val_acc, 'b')
    plt.title('Training and validation accuracy')
    plt.xlabel("Epochs")
    plt.ylabel("Accuracy")
    plt.legend(["Accuracy", "Validation Accuracy"])

    plt.figure()

    #------------------------------------------------
    # Plot training and validation loss per epoch
    #------------------------------------------------
    plt.plot(epochs, loss, 'r')
    plt.plot(epochs, val_loss, 'b')
    plt.title('Training and validation loss')
    plt.xlabel("Epochs")
    plt.ylabel("Loss")
    plt.legend(["Loss", "Validation Loss"])

    plt.figure()


    # Expected Output
    # A chart where the validation loss does not increase sharply!

In [ ]:
plot_h(history)

# Predict

In [ ]:
all_stop_words = list(stop_words)
all_stop_words.extend(check_list)

def preprocess_text(input_text):
    input_text = input_text.lower()
    nltk_tokenizer = nltk.RegexpTokenizer(r"\w+")
    new_words = nltk_tokenizer.tokenize(input_text)
    new_row = [word for word in new_words  if word not in all_stop_words]
    
    final_sentence = " ".join(new_row)
    sequence = tokenizer.texts_to_sequences([final_sentence])
    
    max_length = 320
    trunc_type='post'
    padding_type='post'

    padded = pad_sequences(sequence,maxlen=max_length, padding=padding_type, truncating=trunc_type)
    return padded

In [ ]:
def predict_text(input_text, input_model, label_map):
    padded = preprocess_text(input_text)
    result = input_model.predict(padded)
    #print(result)
    result_index = np.argmax(result)
    predicted_class = label_map[result_index]
    
    return predicted_class

In [ ]:
new_text = "Hola, soy asesor de línea de bebé A Tengo el gusto. Sí, mucho gusto, señorita. Agradecemos su paciencia debido a la contingencia. El número de asesores disponibles reducido. En qué puedo ayudarla? Si mire, lo que pasa es que hace ratito marque. Me dijeron que regresaría la llamada más o menos entre tres y siete de la noche. Por qué? Porque el día de ayer que la reposición de mi tarjeta ese es de la S o la de débito se ahorra. Entonces es que tenía yo en la tarjeta el día de ciento cuarenta y cuatro pesos retenidos hemos sido ocupado para hacer lo del pago porque lo que dicen que en cuanto se hacen depósitos de descuenta lo del pago de la tarjeta correcto. El día de hoy me hicieron donde Pues tío de seiscientos treinta y siete supone que la tarjeta el plástico costaba ciento ochenta. Entonces treinta y siete del depósito veo y los ciento cuarenta y cuatro que tenía que ya con eso completaba hoy Marco para checar mi saldo y me dicen que tengo cuatrocientos noventa y dos. Se supone que tenía que haber quedado seiscientos pesos y quería ver qué es lo que pasó muy bien. Le comento No se activó la tarjeta. El día de hoy me aparece la activación. Bueno, la activación aparece hasta el día de hoy, nueve de septiembre del dos mil veinte. Bueno, ya me entregaron el plato y me dijeron que fuera el casero hacerlo de la activación. Correcto. Muy bien. El día de hoy recibe un abono por seiscientos treinta y siete pesos. Correcto. La tarjeta literalmente estaba en ceros. El día de hoy recibe una buena por seiscientos treinta y siete pesos. Pero si alguien revisa la tarjeta y estaba en silencio ahí, en ciento cuarenta y cuatro, pero estaba el número negativo, es decir, al momento de activarla, se genera el pago, el cobro y el cobro se genera, pero en saldo negativo. Es decir, usted activa la tarjeta y en ese mismo instante, sistema le carga el monto de ciento cuarenta y cuatro presos con sesenta y tres centavos. Saldo negativo que se estaría cobrando el cobro de la tarjeta, que son ciento cuarenta y cinco pesos. Se manejan los ciento cuarenta y cuatro pesos con sesenta y tres centavos aquí. Al parecer, usted tenía un saldo disponible de cero seis centavos. Entonces el cero seis centavos se cobran estos ciento cuarenta y cuatro pesos, con sesenta y tres centavos se quedan en saldo negativo. Porque porque la tarjeta está literalmente en cero. Al momento de que usted le deposita la cantidad de seiscientos treinta y siete pesos, se toman los ciento cuarenta y cuatro punto sesenta y tres para cubrir el cargo del la reposición de tarjeta y usted se queda un saldo con cuatrocientos, noventa y dos pesos con treinta y siete centavos. Muy probablemente usted pensó que tenía los ciento cuarenta y cuatro pesos con sesenta y tres centavos a favor, pero no sé, ya los tenía en saldo negativo porque su cuenta estaba en cero. Es lo que yo tenía duda. Pues sí, era lo que yo tenía ahí de saldo de que ya tiene tiempo de la tarjeta o eran por lo ahora, por lo que se cobraba. Correcto, Es usted. Es decir, usted no tenía ni un solo centavo en bueno, tenía la Navidad de seis centavos en la cuenta, por lo cual pues únicamente le realizan el cargo de cuatro ciento cuarenta y cuatro pesos, con sesenta y tres centavos, ya que el cargo original es de ciento cuarenta y cinco pesos. En este caso se queda el restante. Por los centavos que se toman se queda el mensaje de ciento cuarenta y cuatro pesos, con sesenta y tres centavos de los cuales, como aparecen en saldo negativo, le tuvieron que tuvo que haber aparecido el signo de negativo de menos la pobreza. Saldo negativo. Entonces, al momento de que usted le deposita en la cantidad de seiscientos treinta y siete pesos, pues ese saldo se convierte en positivo, se cobra. Y usted se le descuenta la cantidad de ciento treinta y siete y se queda con el disponible de cuatrocientos noventa y dos punto treinta y siete. Ah, bueno, si es que yo tenía la duda de que bueno, se supone que alguien ciento cuarenta y cuatro. Ciento cuarenta y cuatro. Entonces, qué pasó? Ajá. Pero no está bien si nomas quería quitarme la verdad correcto, señorita, alguna otra información que puede apoyarla? Tendría que ir a tendría que ir al banco si quisiera que me lleguen las notificaciones de mi celular, del, de la de los depósitos. Muy bien, vamos a validar la información. No tiene usted activada la aplicación? Es que que creo que mi celular no me agarra la aplicación. Muy bien. En este caso, necesitaríamos que para que le lleguen las notificaciones o las alertas inicialmente, tener la aplicación móvil para que a partir de ahí se activó en las alertas y le lleguen todas las notificaciones respecto algún cargo, algún cobro, alguna compra por pensar que normal no llegarían. Podría ser con las alertas, señorita. Pero en este caso tendría que ser y activarlas directamente en sucursal. Bueno, sería todo entonces. Por último, su opinión es muy importante para nosotros. En las próximas horas en veremos a su correo electrónico una breve encuesta para calificar el servicio proporcionado en esta llamada. Sus comentando que ayudan a mejorar la atención en cada contacto. Puedo contar con usted, señorita? Bueno, y para poder a nombre de encuesta. Se lo agradecería bastante. Reitero mi nombre es Ismael. Asesor de línea de de uve A. Pase una excelente tarde. Igualmente. Gracias. Hasta luego."

In [ ]:
result = predict_text(new_text, model_prod, index_to_label_prod)
print("RESULTADO:")
print(result)

# Intención

In [ ]:
labels_int = list(df["Intención"])
print(len(labels_int))
n_classes = len(set(labels_int))
print(n_classes)
print(set(labels_int))

In [ ]:
index_to_label_int = {}
label_to_index_int = {}
for i, label in enumerate(set(labels_int)):
    index_to_label_int[i] = label
    label_to_index_int[label] = i
    
print(index_to_label_int)
print(label_to_index_int)

In [ ]:
labels_int_num = np.array([label_to_index_int[label] for label in labels_int])
print(labels_int_num)

In [ ]:
split = int(len(padded)*0.9)
train_x = padded[:split]
valid_x = padded[split:]
train_y = labels_int_num[:split]
valid_y = labels_int_num[split:]
print(len(train_x))
print(len(train_y))

In [ ]:
model_int = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size+1, embedding_dim, input_length=max_length, weights=[embeddings_matrix], trainable=True),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Conv1D(32, 5, activation='relu'), #64->32
    tf.keras.layers.MaxPooling1D(pool_size=4),
    tf.keras.layers.LSTM(16), #64->32
    tf.keras.layers.Dense(n_classes, activation='softmax')
])
model_int.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
model_int.summary()

num_epochs = 30

history = model_int.fit(train_x, train_y, validation_data=(valid_x, valid_y), epochs=num_epochs)

print("Training Complete")

In [ ]:
plot_h(history)

In [ ]:
result = predict_text(new_text, model_int, index_to_label_int)
print("RESULTADO:")
print(result)

# Movimiento

In [ ]:
labels_movimiento = list(df["Tipo de movimiento"])
print(len(labels_movimiento))
n_classes = len(set(labels_movimiento))
print(n_classes)


index_to_label_mov = {}
label_to_index_mov = {}
for i, label in enumerate(set(labels_movimiento)):
    index_to_label_mov[i] = label
    label_to_index_mov[label] = i
    
print(index_to_label_mov)
print(label_to_index_mov)


labels_movimiento_num = np.array([label_to_index_mov[label] for label in labels_movimiento])
print(labels_movimiento_num)


split = int(len(padded)*0.9)
train_x = padded[:split]
valid_x = padded[split:]
train_y = labels_movimiento_num[:split]
valid_y = labels_movimiento_num[split:]
print(len(train_x))
print(len(train_y))


model_mov = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size+1, embedding_dim, input_length=max_length, weights=[embeddings_matrix], trainable=True),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Conv1D(64, 5, activation='relu'), #64->32
    tf.keras.layers.MaxPooling1D(pool_size=4),
    tf.keras.layers.LSTM(32), #64->32
    tf.keras.layers.Dense(n_classes, activation='softmax')
])
model_mov.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
model_mov.summary()

num_epochs = 33

history = model_mov.fit(train_x, train_y, validation_data=(valid_x, valid_y), epochs=num_epochs)

print("Training Complete")

In [ ]:
plot_h(history)

In [ ]:
result = predict_text(new_text, model_mov, index_to_label_mov)
print("RESULTADO:")
print(result)

# Contexto 1

In [ ]:
#### Model_contx1o para contexto 1

labels_contexto = list(df["Contexto 1"])
print(len(labels_contexto))
n_classes = len(set(labels_contexto))
print(n_classes)


index_to_label_contx1 = {}
label_to_index_contx1 = {}
for i, label in enumerate(set(labels_contexto)):
    index_to_label_contx1[i] = label
    label_to_index_contx1[label] = i
    
print(index_to_label_contx1)
print(label_to_index_contx1)


labels_contexto_num = np.array([label_to_index_contx1[label] for label in labels_contexto])
print(labels_contexto_num)


split = int(len(padded)*0.9)
train_x = padded[:split]
valid_x = padded[split:]
train_y = labels_contexto_num[:split]
valid_y = labels_contexto_num[split:]
print(len(train_x))
print(len(train_y))


model_contx1 = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size+1, embedding_dim, input_length=max_length, weights=[embeddings_matrix], trainable=True),
    #tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Conv1D(16, 5, activation='relu'), #64->32
    tf.keras.layers.MaxPooling1D(pool_size=4),
    tf.keras.layers.LSTM(16), #64->32
    tf.keras.layers.Dense(n_classes, activation='softmax')
])
model_contx1.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
model_contx1.summary()

num_epochs = 40

history = model_contx1.fit(train_x, train_y, validation_data=(valid_x, valid_y), epochs=num_epochs)

print("Training Complete")

In [ ]:
plot_h(history)

In [ ]:
result = predict_text(new_text, model_contx1, index_to_label_contx1)
print("RESULTADO:")
print(result)

# Contexto 2

In [ ]:
labels_context_2 = list(df["Contexto 2"])
print(len(labels_context_2))
n_classes = len(set(labels_context_2))
print(n_classes)


index_to_label_contx2 = {}
label_to_index_contx2 = {}
for i, label in enumerate(set(labels_context_2)):
    index_to_label_contx2[i] = label
    label_to_index_contx2[label] = i
    
print(index_to_label_contx2)
print(label_to_index_contx2)


labels_context_2_num = np.array([label_to_index_contx2[label] for label in labels_context_2])
print(labels_context_2_num)


split = int(len(padded)*0.9)
train_x = padded[:split]
valid_x = padded[split:]
train_y = labels_context_2_num[:split]
valid_y = labels_context_2_num[split:]
print(len(train_x))
print(len(train_y))


model_contx2 = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size+1, embedding_dim, input_length=max_length, weights=[embeddings_matrix], trainable=True),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Conv1D(64, 5, activation='relu'), #64->32
    tf.keras.layers.MaxPooling1D(pool_size=4),
    tf.keras.layers.LSTM(32), #64->32
    tf.keras.layers.Dense(n_classes, activation='softmax')
])
model_contx2.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
model_contx2.summary()

num_epochs = 32

history = model_contx2.fit(train_x, train_y, validation_data=(valid_x, valid_y), epochs=num_epochs)

print("Training Complete")

In [ ]:
plot_h(history)

In [ ]:
result = predict_text(new_text, model_contx2, index_to_label_contx2)
print("RESULTADO:")
print(result)

# Detalle 1

In [ ]:
labels_detalle = list(df["Detalle 1"])
print(len(labels_detalle))
n_classes = len(set(labels_detalle))
print(n_classes)


index_to_label_detalle = {}
label_to_index_detalle = {}
for i, label in enumerate(set(labels_detalle)):
    index_to_label_detalle[i] = label
    label_to_index_detalle[label] = i
    
print(index_to_label_detalle)
print(label_to_index_detalle)


labels_detalle_num = np.array([label_to_index_detalle[label] for label in labels_detalle])
print(labels_detalle_num)


split = int(len(padded)*0.9)
train_x = padded[:split]
valid_x = padded[split:]
train_y = labels_detalle_num[:split]
valid_y = labels_detalle_num[split:]
print(len(train_x))
print(len(train_y))


model_detalle = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size+1, embedding_dim, input_length=max_length, weights=[embeddings_matrix], trainable=True),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Conv1D(64, 5, activation='relu'), #64->32
    tf.keras.layers.MaxPooling1D(pool_size=4),
    tf.keras.layers.LSTM(32), #64->32
    tf.keras.layers.Dense(n_classes, activation='softmax')
])
model_detalle.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
model_detalle.summary()

num_epochs = 32

history = model_detalle.fit(train_x, train_y, validation_data=(valid_x, valid_y), epochs=num_epochs)

print("Training Complete")

In [ ]:
plot_h(history)

In [ ]:
result = predict_text(new_text, model_detalle, index_to_label_detalle)
print("RESULTADO:")
print(result)

# save all

In [ ]:
import pickle

#saving model
model_prod.save('model_producto.h5')
model_int.save('model_intencion.h5')
model_mov.save('model_movimiento.h5')
model_contx1.save('model_contexto_1.h5')
model_contx2.save('model_contexto_2.h5')
model_detalle.save('model_detalle.h5')


# saving text tokenizer
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
# saving text tokenizer
with open('all_stop_words.pickle', 'wb') as handle:
    pickle.dump(all_stop_words, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
# saving label map producto
with open('index_to_label_prod.pickle', 'wb') as handle:
    pickle.dump(index_to_label_prod, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
# saving label map intentcion
with open('index_to_label_int.pickle', 'wb') as handle:
    pickle.dump(index_to_label_int, handle, protocol=pickle.HIGHEST_PROTOCOL)

    
# saving label map movimiento
with open('index_to_label_mov.pickle', 'wb') as handle:
    pickle.dump(index_to_label_mov, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
    
# saving label map context 2
with open('index_to_label_contx1.pickle', 'wb') as handle:
    pickle.dump(index_to_label_contx1, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
    
# saving label map context 2
with open('index_to_label_contx2.pickle', 'wb') as handle:
    pickle.dump(index_to_label_contx2, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
# saving label map detalle
with open('index_to_label_detalle.pickle', 'wb') as handle:
    pickle.dump(index_to_label_detalle, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
!zip -r export.zip .